In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset, CHBMITDataModule

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-08 21:09:12.437 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /home/jupyter-group55/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=512, worker=0)

In [ ]:
from typing import List
from eeg_snn_encoder.tuning import filter_and_sort_trials, TrialFilter

be_studies: List[TrialFilter] = [
    {"name": "Classifier_BE_Tuning", "filter_fn": lambda x: x.value < 16.3},
    {"name": "model-tuning-be", "filter_fn": lambda x: x.value < 15},
    {"name": "model-tuning-be-mse", "filter_fn": lambda x: x.value < 0.30},
]

test_trials = filter_and_sort_trials(be_studies)

In [ ]:
import optuna

study = optuna.create_study(
    direction="minimize",
    study_name="model-fine-tuning-be",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    pruner=optuna.pruners.NopPruner()
)

[I 2025-05-08 21:28:15,955] Using an existing study with name 'model-fine-tuning-be' instead of creating a new one.


In [24]:
from loguru import logger

import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback

from eeg_snn_encoder.encoders import BurstEncoder
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig
import torch

torch.set_float32_matmul_precision('medium')


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    max_window = trial.suggest_int("max_window", 4, 16)
    n_max = trial.suggest_int("n_max", 1, max_window)
    t_max = trial.suggest_int("t_max", 0, max_window // n_max)
    t_min = trial.suggest_int("t_min", 0, t_max)

    encoder_params = {
        "max_window": max_window,
        "n_max": n_max,
        "t_max": t_max,
        "t_min": t_min,
    }

    spike_encoder = BurstEncoder(**encoder_params)

    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_mse"), EarlyStopping(monitor="val_loss", mode="min", patience=10)],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    return trainer.callback_metrics["val_mse"]

In [25]:
for trial in test_trials:
    study.enqueue_trial(params=trial.params)

study.optimize(objective, n_trials=len(test_trials))

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-08 22:14:45,987] Trial 39 finished with value: 0.4953519403934479 and parameters: {'threshold': 0.3165862599791414, 'slope': 1.8320709025049458, 'beta': 0.7446333324492806, 'dropout_rate1': 0.2268926646810885, 'dropout_rate2': 0.4444038455342719, 'lr': 4.712094467576679e-05, 'weight_decay': 2.1706538794922772e-06, 'scheduler_factor': 0.5156597729191686, 'scheduler_patience': 10, 'max_window': 1, 'n_max': 4, 't_max': 6, 't_min': 3}. Best is trial 19 with value: 0.16733068227767944.


In [26]:
logger.info(f"Encoder: Burst Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: Burst Encoding,trial, best_score: {study.best_value}")

2025-05-08 22:14:46.461 | INFO     | __main__:<module>:1 - Encoder: Burst Encoding,trial, best_param: {'threshold': 0.07090851207541435, 'slope': 8.107275688930079, 'beta': 0.7440672774049281, 'dropout_rate1': 0.20034293181105056, 'dropout_rate2': 0.5865898197113356, 'lr': 9.362082441139168e-05, 'weight_decay': 1.3936923384285494e-06, 'scheduler_factor': 0.4595170872743502, 'scheduler_patience': 8, 'max_window': 5, 'n_max': 4, 't_max': 0, 't_min': 0}
2025-05-08 22:14:46.938 | INFO     | __main__:<module>:2 - Encoder: Burst Encoding,trial, best_score: 0.16733068227767944
